In [2]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
import itertools
import random
from tqdm import tqdm
sns.set()
import matplotlib.pyplot as plt

# Data

In [5]:
randomized_approx = pickle.load(open('./randomized_approx.pkl', 'rb'))
randomized_approx.head(1)

,year,lp,s1,s2,human_statistic,metric_statistic,human_c,metric_c,human_sig,metric_sig,human_decision,metric_decision
0,2016,cs-en,online-B,uedin-nmt,-0.15642,-0.139296,0,0,True,True,loss,loss


In [58]:
wmt19_toen = pickle.load(open('../7_28-system-level_bleurt/wmt19_toen_bleurt.pkl', 'rb'))
wmt19_toen.head(1)

,system,sid,raw_score,score,N,lp,reference,source,output,year,bleurt
27600,DBMS-KU-KKEN.6726,751,2.0,-1.463163,1,kk-en,Three professional boxers from Kazakhstan will...,Қазақстандық үш кәсіпқой боксшы Мәскеуде жекпе...,three кәсіпқой қазақстандық боксшы өткізеді же...,2019,-1.62813


In [62]:
real_pairs = randomized_approx[randomized_approx.human_sig & (randomized_approx.year == 2019)]
real_pairs.head(1)

,year,lp,s1,s2,human_statistic,metric_statistic,human_c,metric_c,human_sig,metric_sig,human_decision,metric_decision
897,2019,fi-en,parfda.6526,TartuNLP-c.6905,-0.156451,-0.28056,0,0,True,True,loss,loss


In [63]:
sorted_real_pairs = real_pairs.sort_values('human_c')
sorted_real_pairs.tail(10)

,year,lp,s1,s2,human_statistic,metric_statistic,human_c,metric_c,human_sig,metric_sig,human_decision,metric_decision
966,2019,gu-en,UdS-DFKI.6861,IITP-MT.6824,-0.084666,-0.186258,20,0,True,True,loss,loss
1176,2019,ru-en,rerank-re.6540,afrl-syscomb19.6782,0.071311,0.088995,24,0,True,True,gain,gain
1036,2019,kk-en,CUNI-T2T-transfer-kken.6436,NRC-CNRC.6895,-0.089137,-0.031291,28,144,True,False,loss,NS
981,2019,gu-en,UEDIN.6534,NICT.6603,0.081605,0.088744,28,1,True,True,gain,gain
1063,2019,kk-en,rug-kken-morfessor.6677,talp-upc-2019-kken.6657,0.075419,0.064437,31,0,True,True,gain,gain
1195,2019,ru-en,afrl-ewc.6659,online-A.0,-0.068285,-0.060328,33,0,True,True,loss,loss
1134,2019,ru-en,online-Y.0,afrl-syscomb19.6782,0.063652,0.071015,41,0,True,True,gain,gain
1206,2019,ru-en,TartuNLP-u.6650,online-A.0,-0.069097,-0.048746,41,0,True,True,loss,loss
1132,2019,ru-en,online-Y.0,online-G.0,-0.058017,-0.045003,45,4,True,True,loss,loss
1045,2019,kk-en,NICT.6770,online-G.0,-0.068810,-0.099333,48,0,True,True,loss,loss


# Experiments

In [83]:
trials = 100

In [85]:
for i, row in sorted_real_pairs.tail(10).iterrows():
    systems_data = wmt19_toen[(wmt19_toen.lp == row['lp']) & ((wmt19_toen.system == row['s1']) | (wmt19_toen.system == row['s2']))]
    sids = systems_data.sid.unique()
    
    print(row['s1'], row['s2'], 'human_statistic', row['human_statistic'], 'metric_statistic', row['metric_statistic'])
    for proportion in np.linspace(1, 0, 8, endpoint=False):
        total_bleurt, total_human = 0, 0
        for i in range(0, trials):
            chosen = np.random.choice(sids, int(len(sids)*proportion), replace=False)
            B = systems_data[systems_data.sid.isin(chosen)]
            B_scores = B.groupby('system').mean()
            
            bleurt_diff = B_scores['bleurt'][row['s1']] -  B_scores['bleurt'][row['s2']]
            agree_bleurt = np.sign(row['human_statistic']) == np.sign(bleurt_diff)
            total_bleurt += 1 if agree_bleurt else 0
            
            human_diff = B_scores['score'][row['s1']] -  B_scores['score'][row['s2']]
            agree_human = np.sign(row['human_statistic']) == np.sign(human_diff)
            total_human += 1 if agree_human else 0
        print('proportion\t', proportion, '\tbleurt agreement\t', total_bleurt/trials, '\thuman agreement\t', total_human/trials)
    print()    

UdS-DFKI.6861 IITP-MT.6824 human_statistic -0.08466592038335655 metric_statistic -0.18625802826869675
proportion	 1.0 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.875 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.75 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.625 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.5 	bleurt agreement	 1.0 	human agreement	 0.99
proportion	 0.375 	bleurt agreement	 1.0 	human agreement	 0.99
proportion	 0.25 	bleurt agreement	 1.0 	human agreement	 0.92
proportion	 0.125 	bleurt agreement	 1.0 	human agreement	 0.8

rerank-re.6540 afrl-syscomb19.6782 human_statistic 0.07131110180711145 metric_statistic 0.08899520568512326
proportion	 1.0 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.875 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.75 	bleurt agreement	 1.0 	human agreement	 1.0
proportion	 0.625 	bleurt agreement	 1.0 	human agreement	 0.99
proportion	 0.5 	bleurt agreement	 1.0